In [1]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

['/Users/khatran/anaconda3/envs/quantum/lib/python3.11', '/Users/khatran/anaconda3/envs/quantum/lib/python3.11/lib-dynload', '/Users/khatran/anaconda3/envs/quantum/lib/python3.11/site-packages', '/Users/khatran/anaconda3/envs/quantum/lib/tdbcmysql1.1.7', '/Users/khatran/anaconda3/envs/quantum/lib/pkgconfig', '/Users/khatran/anaconda3/envs/quantum/lib/thread2.8.9', '/Users/khatran/anaconda3/envs/quantum/lib/python3.1.c~', '/Users/khatran/anaconda3/envs/quantum/lib/cmake', '/Users/khatran/anaconda3/envs/quantum/lib/tdbcodbc1.1.7', '/Users/khatran/anaconda3/envs/quantum/lib/python3.11', '/Users/khatran/anaconda3/envs/quantum/lib/python3.1', '/Users/khatran/anaconda3/envs/quantum/lib/python3.10', '/Users/khatran/anaconda3/envs/quantum/lib/tk8.6', '/Users/khatran/anaconda3/envs/quantum/lib/tcl8.6', '/Users/khatran/anaconda3/envs/quantum/lib/terminfo', '/Users/khatran/anaconda3/envs/quantum/lib/krb5', '/Users/khatran/anaconda3/envs/quantum/lib/sqlite3.44.2', '/Users/khatran/anaconda3/envs/qu

In [ ]:
# index = sys.path.index('')
# sys.path.pop(index)
path = sys.path[0]
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

for root, dirs, files in os.walk(parent_dir):
    for dir in dirs:
        sys.path.append(os.path.join(root, dir))

sys.path.append(parent_dir)
sys.path.pop(0)
print(sys.path)

In [3]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import inspect

from quantum_optimal_control.helper_functions.grape_functions import *
from quantum_optimal_control.main_grape.grape import Grape

data_path = '../output/x01_π_qutrit/'

ModuleNotFoundError: No module named 'quantum_optimal_control'

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# IBM Quantum channel; set to default
# QiskitRuntimeService.save_account(channel='ibm_quantum', token="0574c8c9cfdbc7c47815e6d5db41423ab35fd457b628bfe1d64b6e9280175fe913a69f1b4c7cdebcdc38a1d843c4200c073954d9f21848cbf9df808b100d17cf", overwrite=True, set_as_default=True)

In [ ]:
# To access saved credentials for the IBM cloud channel
# service = QiskitRuntimeService(channel="ibm_cloud")

# To access saved credentials for the IBM quantum channel
service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
# Specify the instance in service.backend() instead of initializing a new device
backend = service.backend("ibm_kyoto")

In [ ]:
from sympy import *
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import Qubit
from sympy.physics.quantum import Dagger

In [ ]:
ns = 1.0e9
GHz = 1e9
config = backend.configuration()
dt = config.dt
acquire_alignment = config.timing_constraints['acquire_alignment']
granularity = config.timing_constraints['granularity']
pulse_alignment = config.timing_constraints['pulse_alignment']
lcm = np.lcm(acquire_alignment, pulse_alignment)

In [ ]:
def get_closest_multiple_of(value, base_number):
    return int(value + base_number/2) - (int(value + base_number/2) % base_number)

def get_closest_multiple_of_16(num):
    return get_closest_multiple_of(num, granularity)

In [ ]:
def comm(A, B):
  return A*B - B*A

def ket(i):
  if(i == 0):
    return Matrix([[1], [0], [0], [0]])
  if(i == 1):
    return Matrix([[0], [1], [0], [0]])
  if(i == 2):
    return Matrix([[0], [0], [1], [0]])
  if(i == 3):
    return Matrix([[0], [0], [0], [1]])

def bra(i):
  if(i == 0):
    return Matrix([[1, 0, 0, 0]])
  if(i == 1):
    return Matrix([[0, 1, 0, 0]])
  if(i == 2):
    return Matrix([[0, 0, 1, 0]])
  if(i == 3):
    return Matrix([[0, 0, 0, 1]])

def Pi(i):
  return ket(i)*bra(i)

def sigma_x(i, j):
  return ket(j)*bra(i) + ket(i)*bra(j)

def sigma_y(i, j):
  return I*(ket(j)*bra(i) - ket(i)*bra(j))

def sigma_z(i, j):
  return ket(i)*bra(i) - ket(j)*bra(j)

def bch(A, L):
  return A + comm(L, A)

delta2 = -1960219730.9964836/GHz * 2 * np.pi

def delta(j):
  return delta2 * (j - 1) * j / 2

omega01 = 29668641234.851845/GHz

def lambda(j):
  return np.sqrt(j + 1) # /(1 + j * delta2 / (omega - omega01))

In [ ]:
#Defining time scales

Xsteps = 144 #1-2: 160
Ysteps = 144
Zsteps = 1
Xsteps = get_closest_multiple_of_16(Xsteps)
Ysteps = get_closest_multiple_of_16(Ysteps)
# Zsteps = get_closest_multiple_of_16(Zsteps)
total_time = Xsteps * dt * ns #ns dt = 0.2222222222

# Choose optimizing State transfer or Unitary gate
state_transfer = True

#Defining H0
qubit_state_num = 4 
qubit_num = 1 
freq_ge = 29668641234.851845/GHz # Hz

ens = np.array([2 * np.pi * ii * (freq_ge) for ii in np.arange(qubit_state_num)])

# Q_x = np.diag(np.sqrt(np.arange(1, qubit_state_num)), 1) + np.diag(np.sqrt(np.arange(1, qubit_state_num)), -1)
Q_x = lambda(0) * sigma_x(0,1) + lambda(1) * sigma_x(1,2) + lambda(2) * sigma_x(2,3)

# Q_y = (0 + 1j) * (np.diag(np.sqrt(np.arange(1, qubit_state_num)), 1) - np.diag(np.sqrt(np.arange(1, qubit_state_num)), -1))
Q_y = lambda(0) * sigma_y(0,1) + lambda(1) * sigma_y(1,2) + lambda(2) * sigma_y(2,3)

Q_z = np.diag(np.arange(0, qubit_state_num))
Q_z = (1 * Pi(1) + 2 * Pi(2) + 3 * Pi(3))

I_q = np.identity(qubit_state_num)
H_q = np.diag(ens)

H0 = delta(1) * Pi(1) + delta(2) * Pi(2) + delta(3) * Pi(3)

#Defining Concerned states (starting states)
g = np.array([1,0])
e = np.array([0,1])
psi0 = [g,e]

#Defining states to include in the drawing of occupation
states_draw_list = [0,1]
states_draw_names = ['0','1']

#Defining U (Target)
target_vec_0 = np.zeros(qubit_state_num**qubit_num, dtype=complex)
target_vec_0[1] = 1

target_vec_1 = np.zeros(qubit_state_num**qubit_num, dtype=complex)
target_vec_1[0] = 1
    
U = [target_vec_0, target_vec_1]

#Defining U0 (Initial)
q_identity = np.identity(qubit_state_num**qubit_num)
U0 = q_identity

#Defining control Hs
XI = Q_x
YI = Q_y
ZI = Q_z
Hops = [XI, YI, ZI]       

g_ops = 0.5593523210160259 # GHz = Omega_{d, 0}
ops_max_amp = [2 * np.pi * g_ops]
Hnames = ['x']

print(Hnames)

#Defining convergence parameters
max_iterations = 1000
decay = 5000 #max_iterations/2
convergence = {'rate': 0.1, 'update_step': 1, 'max_iterations': max_iterations, 'conv_target': 1e-6, 'learning_rate_decay': decay}

# guassian envelope pulse
reg_coeffs = {'envelope': 0.1, 'dwdt': 0.001}

In [ ]:
uks, U_final = Grape(H0, Hops, Hnames, U, total_time, Xsteps, psi0,
                    convergence = convergence, 
                    draw = [states_draw_list, states_draw_names],
                    state_transfer = True,
                    use_gpu = False,
                    sparse_H = False,
                    show_plots = True, 
                    unitary_error = 1e-6, 
                    method = 'L-BFGS-B', 
                    maxA = ops_max_amp,
                    Taylor_terms = [20,0],
                    reg_coeffs = reg_coeffs,
                    save = True,
                    file_name = 'x01_lagos',
                    data_path = data_path)